## Step 5: Save the Model and Preprocessor
- Save the trained XGBoost model and the PCA object using pickle.
- These files will be used by our Flask API.

## Step 4: Evaluate the Model
- Make predictions on the test set.
- Check the accuracy and other metrics to confirm performance.

## Step 3: Train the XGBoost Model
- [cite_start]Initialize and train the XGBClassifier on the pre-processed data[cite: 167].
- [cite_start]The paper achieved 98.2% accuracy with this model[cite: 16].

## Step 2: Pre-process the Data
- Separate features (X) and the target variable (y).
- [cite_start]Split the data into training (80%) and testing (20%) sets[cite: 133].
- [cite_start]Apply SMOTE to the training data to handle the imbalanced classes[cite: 40, 152].
- [cite_start]Apply PCA for dimensionality reduction[cite: 43, 159].

## Step 1: Import Libraries and Load Data
- Import pandas, SMOTE, PCA, and XGBoost.
- Load the fraud detection dataset using pandas.

# UPI Fraud Detection Model Training

### Project Goal: To build and train an XGBoost model to detect fraudulent transactions based on the provided research paper.

In [8]:
# ===================================================================
# FINAL, COMPLETE MODEL TRAINING SCRIPT
# ===================================================================

# !!! IMPORTANT !!!
# The fraud column name has been corrected below.
# -------------------------------------------------------------------
FRAUD_COLUMN_NAME = 'is_fraud'  # <-- THIS LINE IS NOW CORRECT

# ===================================================================
# (Do not change anything below this line)
# ===================================================================

print("--- Script Starting ---")

# Step 1: Import All Necessary Libraries
try:
    import pandas as pd
    from sklearn.model_selection import train_test_split
    from imblearn.over_sampling import SMOTE
    from sklearn.preprocessing import StandardScaler
    from sklearn.decomposition import PCA
    from xgboost import XGBClassifier
    from sklearn.metrics import accuracy_score, classification_report
    import pickle
    import os
    import numpy as np
    print(f"✅ Step 1: Libraries imported successfully. Using NumPy version: {np.__version__}")
except ImportError as e:
    print(f"❌ ERROR: A required library is not installed. Details: {e}")
    raise

# Step 2: Load the Dataset
try:
    dataset_path = '../creditcard.csv'
    df = pd.read_csv(dataset_path)
    print(f"✅ Step 2: Dataset '{dataset_path}' loaded successfully.")
    if FRAUD_COLUMN_NAME not in df.columns:
        print(f"❌ FATAL ERROR: The column '{FRAUD_COLUMN_NAME}' was not found in your dataset.")
        print(f"    Available columns are: {df.columns.tolist()}")
        raise KeyError(f"Column '{FRAUD_COLUMN_NAME}' not found.")
except FileNotFoundError:
    print(f"❌ FATAL ERROR: 'creditcard.csv' not found in the 'ml_service' folder.")
    raise

# Step 3: Pre-process the Data
print("\n--- Starting Data Pre-processing ---")

# Before we can train the model, we must handle non-numeric data.
# We will select only the numeric columns for this model.
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
# Ensure the fraud column is not accidentally dropped if it's numeric
if FRAUD_COLUMN_NAME in numeric_cols:
     numeric_cols.remove(FRAUD_COLUMN_NAME)

print(f"    > Using numeric columns for training: {numeric_cols}")

# Separate features (X) and the target variable (y) using the variable you set
X = df[numeric_cols]
y = df[FRAUD_COLUMN_NAME]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print("    > Data split into training and testing sets.")

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print("    > Data scaled using StandardScaler.")

print("    > Applying SMOTE to balance the data... (This may take a minute)")
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)
print("    > SMOTE applied successfully.")

pca = PCA(n_components=10) # Using 10 components as there are fewer numeric columns
X_train_pca = pca.fit_transform(X_train_smote)
X_test_pca = pca.transform(X_test_scaled)
print("    > PCA applied for feature extraction.")
print("✅ Step 3: Data pre-processing complete.")

# Step 4: Train the XGBoost Model
print("\n--- Starting Model Training ---")
model = XGBClassifier(objective='binary:logistic', use_label_encoder=False, eval_metric='logloss')
model.fit(X_train_pca, y_train_smote)
print("✅ Step 4: XGBoost model training complete.")

# Step 5: Evaluate the Model
print("\n--- Starting Model Evaluation ---")
y_pred = model.predict(X_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ Step 5: Model evaluation complete. Final Accuracy: {accuracy * 100:.2f}%")
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

# Step 6: Save the Model and Preprocessor
print("\n--- Saving Model and Preprocessor ---")
model_dir = '../model'
os.makedirs(model_dir, exist_ok=True)

model_path = os.path.join(model_dir, 'xgboost_model.pkl')
with open(model_path, 'wb') as f:
    pickle.dump(model, f)
print(f"    > Model saved to: {model_path}")

preprocessor_path = os.path.join(model_dir, 'preprocessor.pkl')
with open(preprocessor_path, 'wb') as f:
    pickle.dump((scaler, pca), f)
print(f"    > Preprocessor saved to: {preprocessor_path}")
print("✅ Step 6: Model and preprocessor saved successfully.")

print("\n🚀 ALL DONE! You can now run your Flask application. 🚀")


--- Script Starting ---
✅ Step 1: Libraries imported successfully. Using NumPy version: 1.26.4
✅ Step 2: Dataset '../creditcard.csv' loaded successfully.

--- Starting Data Pre-processing ---
    > Using numeric columns for training: ['Unnamed: 0', 'cc_num', 'amt', 'zip', 'lat', 'long', 'city_pop', 'unix_time', 'merch_lat', 'merch_long']
    > Data split into training and testing sets.
    > Data scaled using StandardScaler.
    > Applying SMOTE to balance the data... (This may take a minute)
    > SMOTE applied successfully.
    > PCA applied for feature extraction.
✅ Step 3: Data pre-processing complete.

--- Starting Model Training ---


c:\Users\dhanu\anaconda3\envs\final_project_env\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:19:13] WARNING: D:\bld\xgboost-split_1758007502304\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


✅ Step 4: XGBoost model training complete.

--- Starting Model Evaluation ---
✅ Step 5: Model evaluation complete. Final Accuracy: 97.89%

Classification Report:

              precision    recall  f1-score   support

           0       1.00      0.98      0.99    110715
           1       0.13      0.81      0.23       429

    accuracy                           0.98    111144
   macro avg       0.57      0.89      0.61    111144
weighted avg       1.00      0.98      0.99    111144


--- Saving Model and Preprocessor ---
    > Model saved to: ../model\xgboost_model.pkl
    > Preprocessor saved to: ../model\preprocessor.pkl
✅ Step 6: Model and preprocessor saved successfully.

🚀 ALL DONE! You can now run your Flask application. 🚀


In [9]:
import pandas as pd
df = pd.read_csv('../creditcard.csv')
print(df.columns)


Index(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category',
       'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time',
       'merch_lat', 'merch_long', 'is_fraud'],
      dtype='object')
